加载环境变量

In [11]:
from dotenv import load_dotenv
import os
load_dotenv()
# 设置用户代理
os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"

1. 加载文档

In [12]:
from langchain_community.document_loaders import WebBaseLoader # pip install langchain-community

loader = WebBaseLoader(
    web_paths=("https://zh.wikipedia.org/wiki/黑神话：悟空",)
)
docs = loader.load()

2. 文档分块

In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

3. 设置嵌入模型

In [14]:
from langchain_huggingface import HuggingFaceEmbeddings # pip install langchain-huggingface

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-zh-v1.5",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)


4. 创建向量存储

In [15]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)
vector_store.add_documents(all_splits)

['8e583a51-82cc-4aa8-9648-c963e63f519a',
 '2df95fb4-3643-47e5-847f-383ecb0706d0',
 '783cc74b-28b7-4994-be37-5e32ff84c282',
 'e077eabd-b77b-4451-8878-af39957499c5',
 '1f17e3b9-eb71-4f9f-b2e4-bbf742a36b90',
 'd1385609-58af-4a8f-bad9-7e40efc8e887',
 '00d171a2-69fa-4695-9f24-87637a296aa7',
 '87d26357-b5e4-46c7-b384-7f4e20767c86',
 'e6a56f90-efb5-449f-977a-8af55d71d1c2',
 'e1049732-f08a-4bb2-9939-26944a6c57e1',
 'f0939f71-023c-443a-9fc6-974792f8393f',
 '78bdb23a-5b79-4f7c-acb9-b1e6b8b8ef00',
 '6a59fa65-dec9-4396-a7dc-8d72960aaf2d',
 'efcb34c3-39cc-40d4-94ae-970f30bc34dc',
 '6c9ad127-4b95-406c-8c45-5ba070234085',
 'f5e24f18-1563-4e71-800a-ebdf17d24bc1',
 '802aadf2-40a3-4ce7-a607-107daf223a98',
 '43975486-428e-4497-9cbd-d4da8fcb80b6',
 'facca1bc-f7a6-4e06-8502-fdfcbabb065d',
 '47cf61e3-0911-41c6-8015-b2babc929b56',
 '2368e670-abf3-4c07-ac77-8318748c8a9a',
 '4369e756-ef48-4a06-8af0-a617e32103d2',
 'e46cbe23-0890-494f-bcf2-c877096f6750',
 '2e46f7d1-3051-4ed8-ac90-ca1011a2a5fa',
 'de0d0039-b818-

5. 构建用户查询

In [16]:
question = "黑悟空有哪些游戏场景？"

6. 在向量存储中搜索相关文档，并准备上下文内容

In [17]:
retrieved_docs = vector_store.similarity_search(question, k=3)
docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

7. 构建提示模板

In [18]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
                基于以下上下文，回答问题。如果上下文中没有相关信息，
                请说"我无法从提供的上下文中找到相关信息"。
                上下文: {context}
                问题: {question}
                回答:""")

8. 使用大语言模型生成答案

In [19]:
from langchain_deepseek import ChatDeepSeek # pip install langchain-deepseek

llm = ChatDeepSeek(
    model="deepseek-chat",  # DeepSeek API 支持的模型名称
    temperature=0.7,        # 控制输出的随机性
    max_tokens=2048,        # 最大输出长度
    api_key=os.getenv("DEEPSEEK_API_KEY")  # 从环境变量加载API key
)

9. 生成答案

In [20]:
print(f"问题: {question}")
print("-"*50)
answer = llm.invoke(prompt.format(question=question, context=docs_content))
print(f"回答: {answer.content}")

问题: 黑悟空有哪些游戏场景？
--------------------------------------------------
回答: 根据提供的上下文，《黑神话：悟空》的游戏场景融合了中国文化和自然地标，具体包括以下取景地：  
1. **重庆大足石刻**  
2. **山西省的多个地点**：  
   - 小西天  
   - 南禅寺  
   - 铁佛寺  
   - 广胜寺  
   - 鹳雀楼（游戏中标志性地标之一）  
3. 其他未明确列出的场景可能还包括《西游记》原著相关的神话场景（如花果山等），但上下文中未详细提及。  

此外，游戏地图设计为线性与宽阔探索区域结合，并会随剧情进展动态变化，但具体场景名称（如章节中的“火照黑云”“风起黄昏”等）更多属于叙事章节而非地理场景。
